In [16]:
!pip install selenium
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 40 not upgraded.
Need to get 88.3 MB of archives.
After this operation, 294 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 99.0.4844.84-0ubuntu0.18.04.1 [1,142 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 99.0.4844.84-0ubuntu0.18.04.1 [77.7 MB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser-l10n all 99.0.4844.84-0ubuntu0.18.0

In [22]:
import pandas as pd
import numpy as np
import selenium
import time

from geopy.geocoders import Nominatim
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')

In [110]:
df_original = pd.read_csv("/content/data-wrangling-pubs_data_with_crimes.csv", index_col=[0])

In [111]:
df = df_original.copy()

In [112]:
df.head()

,fsq_id,name,address,locality,neighborhood,postcode,category,latitude,longitude,popularity,price,rating,distance_metro,distance_police,number_of_crimes
0,4bc1e42eabf49521c690c193,The Black Horse,92 Wood St,London,Barnet,EN5 4BW,Pub,51.653075,-0.206657,0.939551,1.0,8.1,904.722178,390.281423,30
1,4b995bccf964a5209f7535e3,Ye Olde Mitre Inne,58 High St,Hertfordshire,Barnet,EN5 5SJ,Pub,51.652979,-0.199367,0.982768,1.0,7.6,443.945965,119.561106,45
2,4bb4da3fa7059521b8cc1bce,The Arkley,Barnet Rd,Barnet,Barnet,EN5 3EP,Pub,51.652533,-0.219573,0.987375,1.0,7.7,1767.173867,1215.815963,2
3,4dc436e5ae608779d11bd561,Railway Tavern,3 East Barnet Rd,Barnet,Barnet,EN4 8RR,Pub,51.650059,-0.174692,0.950895,1.0,7.4,1343.248049,1149.621255,64
4,4d72b1e78e12b1f793863f05,The Kings Head,84 High St,Barnet,Barnet,EN5 5SN,Pub,51.653499,-0.200979,0.377777,1.0,6.6,568.292759,98.112834,40


In [113]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1159 entries, 0 to 1158
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   fsq_id            1159 non-null   object 
 1   name              1159 non-null   object 
 2   address           1102 non-null   object 
 3   locality          1134 non-null   object 
 4   neighborhood      989 non-null    object 
 5   postcode          1114 non-null   object 
 6   category          1159 non-null   object 
 7   latitude          1159 non-null   float64
 8   longitude         1159 non-null   float64
 9   popularity        1159 non-null   float64
 10  price             1143 non-null   float64
 11  rating            865 non-null    float64
 12  distance_metro    1159 non-null   float64
 13  distance_police   1159 non-null   float64
 14  number_of_crimes  1159 non-null   int64  
dtypes: float64(7), int64(1), object(7)
memory usage: 144.9+ KB


### Remove rows with null value in Rating

In [114]:
df.isnull().sum()

fsq_id                0
name                  0
address              57
locality             25
neighborhood        170
postcode             45
category              0
latitude              0
longitude             0
popularity            0
price                16
rating              294
distance_metro        0
distance_police       0
number_of_crimes      0
dtype: int64

In [115]:
df.dropna(subset=["rating"], axis=0, inplace=True)

In [117]:
df.shape

(865, 15)

### Filling missing values about price

In [118]:
df["price"].value_counts()

1.0    698
2.0    141
3.0     13
Name: price, dtype: int64

In [119]:
df["price"].fillna(1, inplace=True)

In [120]:
df["price"].value_counts()

1.0    711
2.0    141
3.0     13
Name: price, dtype: int64

### Filling missing values about location info

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 865 entries, 0 to 1157
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   fsq_id            865 non-null    object 
 1   name              865 non-null    object 
 2   address           849 non-null    object 
 3   locality          859 non-null    object 
 4   neighborhood      786 non-null    object 
 5   postcode          860 non-null    object 
 6   category          865 non-null    object 
 7   latitude          865 non-null    float64
 8   longitude         865 non-null    float64
 9   popularity        865 non-null    float64
 10  price             865 non-null    float64
 11  rating            865 non-null    float64
 12  distance_metro    865 non-null    float64
 13  distance_police   865 non-null    float64
 14  number_of_crimes  865 non-null    int64  
dtypes: float64(7), int64(1), object(7)
memory usage: 108.1+ KB


In [122]:
geolocator = Nominatim(user_agent="women_safety_project")

In [123]:
def get_postcode(latitude, longitude):
  geocodes = f"{latitude}, {longitude}"
  location = geolocator.reverse(geocodes)
  location_splitted = location.address.split(",")
  postcode = location_splitted[-2].strip()
  return postcode

In [124]:
empty_postcode = df[df["postcode"].isnull()]

In [125]:
empty_postcode.head()

,fsq_id,name,address,locality,neighborhood,postcode,category,latitude,longitude,popularity,price,rating,distance_metro,distance_police,number_of_crimes
5,4c0bab78ffb8c9b6e9886d61,The Monken Holt,NaN,NaN,Monken Hadley,NaN,Pub,51.658291,-0.201368,0.246146,1.0,6.8,985.031978,313.707805,22
593,4f2188efe4b0cebcfb6406aa,"Ortaköy, Shisha Lounge Bar",83 Mayes Rd,London,NaN,NaN,Pub,51.594699,-0.112428,0.926055,1.0,6.7,338.706696,692.464304,193
674,4c653ed03f69c9b6fea28ad9,The General Elliott,NaN,NaN,NaN,NaN,Pub,51.542912,-0.488453,0.913205,1.0,7.6,890.998637,758.963937,56
808,4c75413c1b30a09338c4ee09,The Limping Fox,Rushett Rd,Long Ditton,NaN,NaN,Pub,51.386031,-0.323829,0.990826,1.0,7.5,8735.604023,1833.711334,0
818,562d09d4498eb208288dda65,The Railway Pub,3 Victoria Rd,London,NaN,NaN,Pub,51.424276,-0.332793,0.916892,1.0,6.9,4847.893502,166.437456,38


In [126]:
empty_postcode["postcode"] = empty_postcode.apply(lambda row: get_postcode(row["latitude"], row["longitude"]), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [128]:
df["postcode"][df["postcode"].isnull()] = empty_postcode["postcode"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [129]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 865 entries, 0 to 1157
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   fsq_id            865 non-null    object 
 1   name              865 non-null    object 
 2   address           849 non-null    object 
 3   locality          859 non-null    object 
 4   neighborhood      786 non-null    object 
 5   postcode          865 non-null    object 
 6   category          865 non-null    object 
 7   latitude          865 non-null    float64
 8   longitude         865 non-null    float64
 9   popularity        865 non-null    float64
 10  price             865 non-null    float64
 11  rating            865 non-null    float64
 12  distance_metro    865 non-null    float64
 13  distance_police   865 non-null    float64
 14  number_of_crimes  865 non-null    int64  
dtypes: float64(7), int64(1), object(7)
memory usage: 140.4+ KB


### Get borough info

In [54]:
postcode_list = df["postcode"].tolist()

In [56]:
browser = webdriver.Chrome(options=options)

In [57]:
borough_list = []
for postcode in postcode_list:
  try:
    browser.get("https://www.gov.uk/find-local-council")
    browser.find_element_by_id("postcode").clear()
    browser.find_element_by_id("postcode").send_keys(postcode)
    browser.find_element_by_css_selector(".gem-c-button.gem-c-button--bottom-margin").click()
    time.sleep(0.5)
    borough = browser.find_element_by_css_selector(".local-authority-results p.govuk-body:first-child").text
    #print(borough)
    borough_list.append(borough)
  except:
    print(postcode)
    borough_list.append(np.nan)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  import sys
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  if __name__ == '__main__':


HA20 2HA
SW6
 SW7
 Sw3


In [130]:
len(borough_list)

865

In [131]:
df.shape

(865, 15)

In [132]:
df["borough"] = borough_list

In [133]:
df.head()

,fsq_id,name,address,locality,neighborhood,postcode,category,latitude,longitude,popularity,price,rating,distance_metro,distance_police,number_of_crimes,borough
0,4bc1e42eabf49521c690c193,The Black Horse,92 Wood St,London,Barnet,EN5 4BW,Pub,51.653075,-0.206657,0.939551,1.0,8.1,904.722178,390.281423,30,London Borough of Barnet
1,4b995bccf964a5209f7535e3,Ye Olde Mitre Inne,58 High St,Hertfordshire,Barnet,EN5 5SJ,Pub,51.652979,-0.199367,0.982768,1.0,7.6,443.945965,119.561106,45,London Borough of Barnet
2,4bb4da3fa7059521b8cc1bce,The Arkley,Barnet Rd,Barnet,Barnet,EN5 3EP,Pub,51.652533,-0.219573,0.987375,1.0,7.7,1767.173867,1215.815963,2,London Borough of Barnet
3,4dc436e5ae608779d11bd561,Railway Tavern,3 East Barnet Rd,Barnet,Barnet,EN4 8RR,Pub,51.650059,-0.174692,0.950895,1.0,7.4,1343.248049,1149.621255,64,London Borough of Barnet
4,4d72b1e78e12b1f793863f05,The Kings Head,84 High St,Barnet,Barnet,EN5 5SN,Pub,51.653499,-0.200979,0.377777,1.0,6.6,568.292759,98.112834,40,London Borough of Barnet


In [134]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 865 entries, 0 to 1157
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   fsq_id            865 non-null    object 
 1   name              865 non-null    object 
 2   address           849 non-null    object 
 3   locality          859 non-null    object 
 4   neighborhood      786 non-null    object 
 5   postcode          865 non-null    object 
 6   category          865 non-null    object 
 7   latitude          865 non-null    float64
 8   longitude         865 non-null    float64
 9   popularity        865 non-null    float64
 10  price             865 non-null    float64
 11  rating            865 non-null    float64
 12  distance_metro    865 non-null    float64
 13  distance_police   865 non-null    float64
 14  number_of_crimes  865 non-null    int64  
 15  borough           861 non-null    object 
dtypes: float64(7), int64(1), object(8)
memory u

In [135]:
df[df["borough"].isnull()]

,fsq_id,name,address,locality,neighborhood,postcode,category,latitude,longitude,popularity,price,rating,distance_metro,distance_police,number_of_crimes,borough
47,4bd7c86a304fce72600534ab,The Fusilier,Harrow Rd,London,Sudbury,HA20 2HA,Pub,51.561739,-0.331929,0.960519,1.0,7.2,595.956601,2138.848420,30,NaN
551,57ae03c938fa6c46b7e49fa1,McGettigan's Fulham,1 Fulham Broadway,London,NaN,SW6,Pub,51.479888,-0.196053,0.955144,1.0,6.7,94.942344,273.991747,164,NaN
571,4b7c3a5af964a520d1852fe3,Gloucester Arms,Victoria Grove,Greater London,NaN,SW7,Arts and Entertainment,51.498752,-0.183882,0.960377,1.0,5.9,612.484689,978.313212,108,NaN
760,4bc22283abf49521cf06c293,The Sydney Arms,Sydney St,Chelsea,NaN,Sw3,Pub,51.488642,-0.169781,0.946835,1.0,6.8,638.109791,437.177332,111,NaN


In [136]:
df["borough"][df["postcode"] == "HA20 2HA"] = "London Borough of Brent"
df["postcode"][df["postcode"] == "HA20 2HA"] = "HA0 2HA"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [137]:
df["borough"][df["postcode"] == "SW6"] = "London Borough of Hammersmith & Fulham"
df["postcode"][df["postcode"] == "SW6"] = "SW6 1AA"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [138]:
df["borough"][df["postcode"] == " SW7"] = "Royal Borough of Kensington and Chelsea"
df["postcode"][df["postcode"] == " SW7"] = "SW7 4RB"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [139]:
df["borough"][df["postcode"] == " Sw3"] = "Royal Borough of Kensington and Chelsea"
df["postcode"][df["postcode"] == " Sw3"] = "SW3 6NJ"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [140]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 865 entries, 0 to 1157
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   fsq_id            865 non-null    object 
 1   name              865 non-null    object 
 2   address           849 non-null    object 
 3   locality          859 non-null    object 
 4   neighborhood      786 non-null    object 
 5   postcode          865 non-null    object 
 6   category          865 non-null    object 
 7   latitude          865 non-null    float64
 8   longitude         865 non-null    float64
 9   popularity        865 non-null    float64
 10  price             865 non-null    float64
 11  rating            865 non-null    float64
 12  distance_metro    865 non-null    float64
 13  distance_police   865 non-null    float64
 14  number_of_crimes  865 non-null    int64  
 15  borough           865 non-null    object 
dtypes: float64(7), int64(1), object(8)
memory u

#Select variables for analysis

In [142]:
df.drop(columns=["address", "locality", "neighborhood", "category", "fsq_id"], inplace=True)

In [143]:
df.head()

,name,postcode,latitude,longitude,popularity,price,rating,distance_metro,distance_police,number_of_crimes,borough
0,The Black Horse,EN5 4BW,51.653075,-0.206657,0.939551,1.0,8.1,904.722178,390.281423,30,London Borough of Barnet
1,Ye Olde Mitre Inne,EN5 5SJ,51.652979,-0.199367,0.982768,1.0,7.6,443.945965,119.561106,45,London Borough of Barnet
2,The Arkley,EN5 3EP,51.652533,-0.219573,0.987375,1.0,7.7,1767.173867,1215.815963,2,London Borough of Barnet
3,Railway Tavern,EN4 8RR,51.650059,-0.174692,0.950895,1.0,7.4,1343.248049,1149.621255,64,London Borough of Barnet
4,The Kings Head,EN5 5SN,51.653499,-0.200979,0.377777,1.0,6.6,568.292759,98.112834,40,London Borough of Barnet


In [144]:
columns_order = ["name", "postcode", "borough", "latitude", "longitude", "price", "rating", "popularity", "distance_metro", "distance_police", "number_of_crimes"]
df = df.reindex(columns=columns_order)

In [145]:
df.head()

,name,postcode,borough,latitude,longitude,price,rating,popularity,distance_metro,distance_police,number_of_crimes
0,The Black Horse,EN5 4BW,London Borough of Barnet,51.653075,-0.206657,1.0,8.1,0.939551,904.722178,390.281423,30
1,Ye Olde Mitre Inne,EN5 5SJ,London Borough of Barnet,51.652979,-0.199367,1.0,7.6,0.982768,443.945965,119.561106,45
2,The Arkley,EN5 3EP,London Borough of Barnet,51.652533,-0.219573,1.0,7.7,0.987375,1767.173867,1215.815963,2
3,Railway Tavern,EN4 8RR,London Borough of Barnet,51.650059,-0.174692,1.0,7.4,0.950895,1343.248049,1149.621255,64
4,The Kings Head,EN5 5SN,London Borough of Barnet,51.653499,-0.200979,1.0,6.6,0.377777,568.292759,98.112834,40


In [146]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 865 entries, 0 to 1157
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   name              865 non-null    object 
 1   postcode          865 non-null    object 
 2   borough           865 non-null    object 
 3   latitude          865 non-null    float64
 4   longitude         865 non-null    float64
 5   price             865 non-null    float64
 6   rating            865 non-null    float64
 7   popularity        865 non-null    float64
 8   distance_metro    865 non-null    float64
 9   distance_police   865 non-null    float64
 10  number_of_crimes  865 non-null    int64  
dtypes: float64(7), int64(1), object(3)
memory usage: 113.4+ KB


In [148]:
df.to_csv("data-wrangling-pubs-data-complete.csv")